<a href="https://colab.research.google.com/github/ZA3karia/course-articles-videos-recomendation/blob/master/webscraping_indeed_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install newspaper3k

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 194kB 44.2MB/s 
     |████████████████████████████████| 7.4MB 38.5MB/s 
     |████████████████████████████████| 51kB 14.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
Successfully built feedparser tinysegmenter jieba3k feedfinder2


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 18 14:47:12 2019

@author: ZAZA
"""

import re
import json
from bs4 import BeautifulSoup
from time import sleep
import requests
# be kind to the server - set num seconds to delay prior to
# fetching each URL
FETCH_DELAY_SECONDS = 5 # or whatever value you're comfortable with
from newspaper import Article
from pandas import DataFrame


def get_soup(url):
   sleep(FETCH_DELAY_SECONDS)
   req = requests.get(url)
   soup = BeautifulSoup(req.text, 'html.parser') 
   return soup



def grab_job_links(soup):
    """
    Grab all non-sponsored job posting links from a Indeed search result page using the given soup object
    
    Parameters:
        soup: the soup object corresponding to a search result page
                e.g. https://ca.indeed.com/jobs?q=data+scientist&l=Toronto&start=20
    
    Returns:
        urls: a python list of job posting urls
    
    """
    i=0
    urls = []
    for div in soup.find_all('div', {'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
      for link in div.find_all('div',{'class': 'title'}):
        i+=1
        try:
          partial_url = link.a.get('href')
          url = 'https://www.indeed.com' + partial_url  
          if i%2==0:
            urls.append(url)
            print(i/2)
        except:
          continue
    
    return urls



def get_urls(query, num_pages, location):
    """
    Get all the job posting URLs resulted from a specific search.
    
    Parameters:
        query: job title to query
        num_pages: number of pages needed
        location: city to search in
    
    Returns:
        urls: a list of job posting URL's (when num_pages valid)
        max_pages: maximum number of pages allowed ((when num_pages invalid))
    """
    # We always need the first page
    base_url = 'https://www.indeed.com/jobs?q={}&l={}'.format(query, location)
    soup = get_soup(base_url)
    urls = []
    urls.append( grab_job_links(soup))
    """
    # Get the total number of postings found 
    posting_count_string = soup.find(name='div', attrs={'id':"searchCount"}).text
    posting_count_string = posting_count_string[posting_count_string.find('of')+2:].strip()
    print('posting_count_string: {}'.format(posting_count_string))
    #print('type is: {}'.format(type(posting_count_string)))
    
    try:
        posting_count = int(posting_count_string)
    except ValueError: # deal with special case when parsed string is "360 jobs"
        posting_count = int(re.search('\d+', posting_count_string).group(0))
        #print('posting_count: {}'.format(posting_count))
        #print('\ntype: {}'.format(type(posting_count)))
    finally:
        posting_count = 330 # setting to 330 when unable to get the total
        pass
    
    #posting_count = 200
    # Limit nunmber of pages to get
    max_pages = round(posting_count / 10) - 3
    if num_pages > max_pages:
        print('returning max_pages!!')
        num_pages = max_pages
    """
        # Additional work is needed when more than 1 page is requested
    if num_pages >= 2:
        # Start loop from page 2 since page 1 has been dealt with above
        for i in range(2, num_pages+1):
            num = (i-1) * 10
            base_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(query, location, num)
            print(base_url)
            
            soup = get_soup(base_url)
            # We always combine the results back to the list
            #print("current urls state :" + grab_job_links(soup)[0])
            urls.append(grab_job_links(soup))
                
            """
            
            try:
                soup = get_soup(base_url)
                # We always combine the results back to the list
                print("current urls state :" + grab_job_links(soup))
                urls.append(grab_job_links(soup))
                
            except:
                continue
            """

    # Check to ensure the number of urls gotten is correct
    #assert len(urls) == num_pages * 10, "There are missing job links, check code!"

    return urls     



def get_posting(url):
    """
    Get the text portion including both title and job description of the job posting from a given url
    
    Parameters:
        url: The job posting link
        
    Returns:
        title: the job title (if "data scientist" is in the title)
        posting: the job posting content    
    """
    # Get the url content as BS object
    soup = get_soup(url)
    
    # The job title is held in the h3 tag
    title = soup.find(name='h3').getText().lower()
    posting = soup.find(name='div', attrs={'class': "jobsearch-JobComponent"}).get_text()

    return title, posting.lower()

        
    #if 'data scientist' in title:  # We'll proceed to grab the job posting text if the title is correct
        # All the text info is contained in the div element with the below class, extract the text.
        #posting = soup.find(name='div', attrs={'class': "jobsearch-JobComponent"}).get_text()
        #return title, posting.lower()
    #else:
        #return False
    
        # Get rid of numbers and symbols other than given
        #text = re.sub("[^a-zA-Z'+#&]", " ", text)
        # Convert to lower case and split to list and then set
        #text = text.lower().strip()
    
        #return text
def get_art(url):
  article = Article(url)
  article.download()
  article.parse()
  article.nlp()
  title = article.title
  text = article.text
  return title,text

def get_data(query, num_pages, location='usa'):
    """
    Get all the job posting data and save in a json file using below structure:
    
    {<count>: {'title': ..., 'posting':..., 'url':...}...}
    
    The json file name has this format: ""<query>.json"
    
    Parameters:
        query: Indeed query keyword such as 'Data Scientist'
        num_pages: Number of search results needed
        location: location to search for
    
    Returns:
        postings_dict: Python dict including all posting data
    
    """
    # Convert the queried title to Indeed format
    query = '+'.join(query.lower().split())
    
    #postings_dict = {}
    tab_url = []
    tab_titles = []
    tab_post = []
    urls = get_urls(query, num_pages, location)
    print("---------------------------------------------part1-------------------------------------------------")
    #  Continue only if the requested number of pages is valid (when invalid, a number is returned instead of list)
    i=0
    print("----------------------------urls:" )
    print(urls)
    for page_urls in urls:
      
      print("page urls :" )
      print(page_urls)
      for url in page_urls:
        print("url")
        print( url)
        try:
            #title, posting = get_posting(url)
            title, posting = get_art(url)
            tab_url.append(url)
            tab_titles.append(title)
            tab_post.append(posting)
            #postings_dict[i] = {}
            #postings_dict[i]['title'], postings_dict[i]['posting'], postings_dict[i]['url'] = title, posting, url
            postings_table.append(post)
            #i+=1
        except: 
            continue
        """
        
        percent = (i+1) / num_urls
        # Print the progress the "end" arg keeps the message in the same line 
        print("Progress: {:2.0f}%".format(100*percent), end='\r')

        """
    posting_dict = {'title' : tab_titles,
                    'url' : tab_url,
                    'post' : tab_post
    }
    # Save the dict as json file
    file_name = query.replace('+', '_') + '.json'
    with open(file_name, 'w') as f:
        json.dump(posting_dict, f)

    print('All {} postings have been scraped and saved!'.format(i))    
    
    df = DataFrame(posting_dict, columns = ['title','url','post'])
    file_name = query.replace('+', '_') + '.csv'
    df.to_csv( file_name)
    return posting_dict
  


# If script is run directly, we'll take input from the user
if __name__ == "__main__":
    
    query = input("Please enter the title to scrape data for: \n").lower()
    
    while True:
        num_pages = input("Please enter the number of pages needed (integer only): \n")
        try:
            num_pages = int(num_pages)
            break
        except:
            print("Invalid number of pages! Please try again.")

    get_data(query, num_pages, location='usa')


Please enter the title to scrape data for: 
web dev
Please enter the number of pages needed (integer only): 
300
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
posting_count_string: 2,634 jobs
returning max_pages!!
https://www.indeed.com/jobs?q=web+dev&l=usa&start=10
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=20
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=30
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=40
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=50
1.0
2.0
3.0
4.0
5.0
6.0
7.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=60
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=70
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=80
1.0
2.0
3.0
4.0
5.0
6.0
7.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=90
1.0
2.0
3.0
4.0
5.0
6.0
7.0
https://www.indeed.

In [0]:
get_data("web dev", 10, location='usa')

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
posting_count_string: 2,637 jobs
https://www.indeed.com/jobs?q=web+dev&l=usa&start=10
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=20
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=30
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=40
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=50
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=60
1.0
2.0
3.0
4.0
5.0
6.0
7.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=70
1.0
2.0
3.0
4.0
5.0
6.0
7.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=80
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
https://www.indeed.com/jobs?q=web+dev&l=usa&start=90
1.0
2.0
3.0
4.0
5.0
6.0
7.0
All 0 postings have been scraped and saved!


{}

In [0]:
url = "https://www.indeed.com/jobs?q=web+dev&l=usa&start=10"
soup = get_soup(url)

In [0]:
def grab_job_links(soup):
    """
    Grab all non-sponsored job posting links from a Indeed search result page using the given soup object
    
    Parameters:
        soup: the soup object corresponding to a search result page
                e.g. https://ca.indeed.com/jobs?q=data+scientist&l=Toronto&start=20
    
    Returns:
        urls: a python list of job posting urls
    
    """
    i=0
    urls = []
    for div in soup.find_all('div', {'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
      for link in div.find_all('div',{'class': 'title'}):
        i+=1
        try:
          partial_url = link.a.get('href')
                 
          if i%2==0:
            urls.append(partial_url)
            print(i/2)
        except:
          continue
    
    return urls

In [0]:
i=0
urls = []
for div in soup.find_all('div', {'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
  for link in div.find_all('div',{'class': 'title'}):
    print(i)

    i+=1
    try:
      partial_url = link.a.get('href')
      print(partial_url)
      #print(link)
      if i%2==0:
        urls.append(partial_url)
    except:
      continue

0
/pagead/clk?mo=r&ad=-6NYlbfkN0DerswQM2HhQOfmrvsK2RVQziLxhA37FaTLzJW8U6ozItnu-fKqGafU4_7xwcz3HSNd2Yybtw2JMvDhvQFOMI5-1dBPt09R0rDpERylyZ6_umFs0oos6JDNgVjgAz0aTriRc00O0PCF6NmiOaeldNNW929wHIjkjEW99c-9Kqgr6bHWzeljl84kT_-_wWcI29P7iXHicoN8Ay_ySUeDWkugKggzs0ELbnTOmLHqiFYKaLkpkDyHXQSCok8kNtWHhnWGjLKFetRHr_pZaty5YTnngyEZGOClDAdG_ZPPF944bscqptg3UofwLww003fUgdNm6TPJV73lp-w4YtSj1o-FxRc_1TG3_ZZSyfhNPGMHxBmyAC7-fscD4NGraOuK1TvxvB8j7-JiREtLkG8CZOmExZVLwQNCJP4HZ4s-MEkM14DPerhI2Vat1joe11ENaqI=&p=0&fvj=1&vjs=3
1
/pagead/clk?mo=r&ad=-6NYlbfkN0C_fyVulV5iV-kRLlyl9iug2ZDt9XqbFBKZixmw1InbSuIZP82twKJRruBF6Yw9-BeRaFK1m_JWmfMHgnCC1fhzdvw4q3wqOsbkCbcLi5PtbhRSHosUzKpQTXyb1VHfD9xzGCxWB3u7pbfXksw9Pjx7J2B4FP1-XufogVYOmzi9ywiILI2sr3x6l3X4G1cAUSJA9G7KSBuZhDwfFXlUWeHyWtTRt7eWeSLXjNxrw5ptT-Ngnd-xlfhMoUMRluuGYHgt4zqJt-E9IFgqkCtSd087GbJa_f9Cm2GCXkb6CfbUHzrNn6XDQvCs7DrjteIIQdfrjI3V0iguIoWB8BAp8qgRa34dFFcEJhNtB-GzH_Tn5wqk8-5NBb93Zh7trPX0Se6yB78XKRU2gELuDwBah9c99ol2fUspwr5ZAyuDlg4Wtg==&p=1&fvj=1&vjs=3
2
/pagead/clk?mo=r&ad=

In [0]:
urls


['/pagead/clk?mo=r&ad=-6NYlbfkN0C_fyVulV5iV-kRLlyl9iug2ZDt9XqbFBKZixmw1InbSuIZP82twKJRruBF6Yw9-BeRaFK1m_JWmfMHgnCC1fhzdvw4q3wqOsbkCbcLi5PtbhRSHosUzKpQTXyb1VHfD9xzGCxWB3u7pbfXksw9Pjx7J2B4FP1-XufogVYOmzi9ywiILI2sr3x6l3X4G1cAUSJA9G7KSBuZhDwfFXlUWeHyWtTRt7eWeSLXjNxrw5ptT-Ngnd-xlfhMoUMRluuGYHgt4zqJt-E9IFgqkCtSd087GbJa_f9Cm2GCXkb6CfbUHzrNn6XDQvCs7DrjteIIQdfrjI3V0iguIoWB8BAp8qgRa34dFFcEJhNtB-GzH_Tn5wqk8-5NBb93Zh7trPX0Se6yB78XKRU2gELuDwBah9c99ol2fUspwr5ZAyuDlg4Wtg==&p=1&fvj=1&vjs=3',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0Dx6RNK3oI658vq4-fl1cPO0MaL36gW1qNhHGqaWSoNWxPaZEwOvsrAw08yZAtE2yQj8T5hEZNiGPs6k_USl9W9iNWLfuHrC_P9uSHt_7p8Hu3APPb2ZBxGhOxJBlk7TDg9JCmFr6sSjsg2ogbD0Sf3VwEcDsZbgy86r_e5M-BJs3q7UuaLzsjIMGYVVnHyNd50l3z75eIG8MJ6XtWLw_ePUWB5YBKysyfWNHuCsFcj0yLeOy8e5dzLStU1zw-M55OSfG53y7myUE0M7dYsw5tQLP4IHT6qg4OhdrKihLuZJnow7GK9mmoq0hIn5Ju-ljoZxiTUm_WxzsFCzraZPUJcDoVF3lLU2ZC50wfVShsX9Q-hze1UUgeKs_JdMmh_8-9zngISr01gN8QMvAR5AVgtu-x7F3yUXpLJSsT_3boSofXn__HSm23UcDBMfgzz9M75SFcQqltU5rneHs6eUgh0odlBlkgdHhwz4I3Xr

# New Section